In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
from matplotlib import style
from sklearn import preprocessing
import warnings 
warnings.filterwarnings('ignore')
style.use('ggplot')

In [18]:
df_train = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')
df_train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [5]:
df_train.shape

(381109, 12)

In [6]:
df_test.shape

(127037, 11)

In [10]:
df_train.describe(include = 'all')

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,381109.000000,381109,381109.000000,381109.000000,381109.000000,381109.000000,381109,381109,381109.000000,381109.000000,381109.000000,381109.000000
unique,NaN,2,NaN,NaN,NaN,NaN,3,2,NaN,NaN,NaN,NaN
top,NaN,Male,NaN,NaN,NaN,NaN,1-2 Year,Yes,NaN,NaN,NaN,NaN
freq,NaN,206089,NaN,NaN,NaN,NaN,200316,192413,NaN,NaN,NaN,NaN
mean,190555.000000,NaN,38.822584,0.997869,26.388807,0.458210,NaN,NaN,30564.389581,112.034295,154.347397,0.122563
std,110016.836208,NaN,15.511611,0.046110,13.229888,0.498251,NaN,NaN,17213.155057,54.203995,83.671304,0.327936
min,1.000000,NaN,20.000000,0.000000,0.000000,0.000000,NaN,NaN,2630.000000,1.000000,10.000000,0.000000
25%,95278.000000,NaN,25.000000,1.000000,15.000000,0.000000,NaN,NaN,24405.000000,29.000000,82.000000,0.000000
50%,190555.000000,NaN,36.000000,1.000000,28.000000,0.000000,NaN,NaN,31669.000000,133.000000,154.000000,0.000000
75%,285832.000000,NaN,49.000000,1.000000,35.000000,1.000000,NaN,NaN,39400.000000,152.000000,227.000000,0.000000


In [3]:
%%time
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB
Wall time: 496 ms


**Data prepartion**

**Categorical Feature Generating**

In [19]:
cat_feats = ['Gender' , 'Vehicle_Age', 'Vehicle_Damage' ]

import category_encoders as ce 

target_enc = ce.TargetEncoder(cols = cat_feats)
target_enc.fit(df_train[cat_feats] , df_train['Response'])
train_te = target_enc.transform(df_train[cat_feats])
test_te =  target_enc.transform(df_test[cat_feats])
df_train = df_train.join(train_te.add_suffix('_target'))
df_test = df_test.join(test_te.add_suffix('_target'))
df_train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Gender_target,Vehicle_Age_target,Vehicle_Damage_target
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1,0.138411,0.293746,0.237655
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0,0.138411,0.173755,0.005204
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1,0.138411,0.293746,0.237655
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0,0.138411,0.043705,0.005204
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0,0.103902,0.043705,0.005204


In [20]:
target_cbe = ce.CatBoostEncoder(cols = cat_feats)
target_cbe.fit(df_train[cat_feats] , df_train['Response'])
train_cbe = target_cbe.transform(df_train[cat_feats])
test_cbe = target_cbe.transform(df_test[cat_feats])
df_train = df_train.join(train_cbe.add_suffix('_cb'))
df_test = df_test.join(test_cbe.add_suffix('_cb'))
df_train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,Gender_target,Vehicle_Age_target,Vehicle_Damage_target,Gender_cb,Vehicle_Age_cb,Vehicle_Damage_cb
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1,0.138411,0.293746,0.237655,0.138411,0.293736,0.237655
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0,0.138411,0.173755,0.005204,0.138411,0.173755,0.005205
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1,0.138411,0.293746,0.237655,0.138411,0.293736,0.237655
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0,0.138411,0.043705,0.005204,0.138411,0.043706,0.005205
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0,0.103902,0.043705,0.005204,0.103903,0.043706,0.005205


In [21]:
count_enc = ce.CountEncoder()
count_encoded =  count_enc.fit_transform(df_train[cat_feats])
df_train = df_train.join(count_encoded.add_suffix('_count'))
count_encoded =  count_enc.fit_transform(df_test[cat_feats])
df_test = df_test.join(count_encoded.add_suffix('_count'))

df_train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,...,Response,Gender_target,Vehicle_Age_target,Vehicle_Damage_target,Gender_cb,Vehicle_Age_cb,Vehicle_Damage_cb,Gender_count,Vehicle_Age_count,Vehicle_Damage_count
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,...,1,0.138411,0.293746,0.237655,0.138411,0.293736,0.237655,206089,16007,192413
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,...,0,0.138411,0.173755,0.005204,0.138411,0.173755,0.005205,206089,200316,188696
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,...,1,0.138411,0.293746,0.237655,0.138411,0.293736,0.237655,206089,16007,192413
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,...,0,0.138411,0.043705,0.005204,0.138411,0.043706,0.005205,206089,164786,188696
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,...,0,0.103902,0.043705,0.005204,0.103903,0.043706,0.005205,175020,164786,188696


In [22]:
df_test['Response'] = -1

In [23]:
df = pd.concat([df_train , df_test])
df.drop(['id' , 'Region_Code' ] ,  axis = 1, inplace = True)

In [24]:
df.shape

(508146, 19)

** Numerical Feature Generating**

In [34]:
# Converting Categorical data into numerical

def Categorical_into_Numerical(df):
    columns = df.columns.values
    
    for column in columns:
        text_digit_values = {}

        def convert_into_int(val):
            return text_digit_values[val]

        if df[column].dtype == np.object:
            column_content = df[column].values.tolist()
            unique_content = set(column_content)
            x = 0
            for unique in unique_content:
                if unique not in text_digit_values:
                    text_digit_values[unique] = x
                    x+=1

            df[column] = list(map(convert_into_int , df[column]))
    return df

df = Categorical_into_Numerical(df)
df.head()

,Gender,Driving_License,Previously_Insured,Vehicle_Age,Vehicle_Damage,Policy_Sales_Channel,Vintage,Response,Daily_Premium,Age_Cat
0,0,1,0,2,0,26.0,217,1,186.423963,4
1,0,1,0,0,1,26.0,183,0,183.256831,5
2,0,1,0,2,0,26.0,27,1,1418.296296,4
3,0,1,1,1,1,152.0,203,0,140.980296,0
4,1,1,1,1,1,152.0,39,0,705.025641,0


**Standardization of Nnumerical feature**

In [37]:
def Scaling_Numerical_Feature(df , num_cols):
    scaler = preprocessing.StandardScaler()
    for col in num_cols:
        col_content = df[col].values
        col_content = col_content.reshape(len(df) , -1)
        scale_content = scaler.fit_transform(col_content)
        df[col] = scale_content

    return df
num_cols = ['Policy_Sales_Channel' ,'Vintage' , 'Daily_Premium' ]
df_scale = Scaling_Numerical_Feature(df ,  num_cols)
df_scale.head()

,Gender,Driving_License,Previously_Insured,Vehicle_Age,Vehicle_Damage,Policy_Sales_Channel,Vintage,Response,Daily_Premium,Age_Cat
0,0,1,0,2,0,-1.584926,0.748905,1,-0.323944,4
1,0,1,0,0,1,-1.584926,0.342540,0,-0.329729,5
2,0,1,0,2,0,-1.584926,-1.521956,1,1.925952,4
3,0,1,1,1,1,0.737827,0.581578,0,-0.406943,0
4,1,1,1,1,1,0.737827,-1.378534,0,0.623232,0


In [38]:
df_scaled_test = df_scale[df_scale['Response'] == -1]
df_scaled_train = df_scale[((df_scale['Response'] == 1) | (df_scale['Response'] == 0))]
df_scaled_test.drop(['Response'] , 1, inplace = True)

In [39]:
print(df_scaled_train.shape)
print(df_scaled_test.shape)

(381109, 10)
(127037, 9)


In [40]:
x_feature = df_scaled_train.drop(['Response'] , 1 )
y_response = df_scaled_train['Response']

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

x_train ,x_val , y_train, y_val = train_test_split(x_feature, y_response, test_size = 0.2)

log_clf = LogisticRegression()
log_clf.fit(x_train , y_train)

y_pred = log_clf.predict(x_val)

roc_auc_score(y_val , y_pred)

0.5

In [48]:
%%time
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()

tree_clf.fit(x_train, y_train)
y_pred = tree_clf.predict(x_val)
roc_auc_score(y_val , y_pred)

Wall time: 3.02 s


0.5971352596459489

In [50]:
pip install XGBoost

Note: you may need to restart the kernel to use updated packages.


In [51]:
from XGBoost import XGBlassifier

ModuleNotFoundError: No module named 'XGBoost'

In [68]:
sub_sample = pd.read_csv('./dataset/sample_submission.csv')

In [69]:
y_pred = tree_clf.predict(df_scaled_test.values)

In [70]:
sub_sample['Response'] = y_pred

In [71]:
sub_sample.to_csv('./dataset/sample_sub1.0.csv', index = False)